---
layout: post
title: Tri 2 Retrospective
description: Final Tri 2
permalink: /final/
comments: true
---

# Retrospective from Tri 2, Final

## 5 Things I Did This Trimester (as a deployment admin)

- 1.) Static API Data
    - Coordinated team efforts to establish and test API endpoints prior to full deployment, and verified that they worked using Postman
- 2.) Dynamic Backend Data
    - Helped troubleshoot database requests and enhance performance efficiency.
- 3.) CRUD Operations
    - Developed and integrated essential CRUD (Create, Read, Update, Delete) operations to improve data handling.
    - Verified operations through Postman
- 4.) Deploment Assistant Admin
    - Tasked with organizing deployment into a burndown list that helped to guide my teammates towards success
- 5.) Collaboration
    - Developed collaborative relationships with peers from my period and other CSP classes in order to succeed

## Project Feature Blog Write-Up

- **Message Input/Output**: Handling user-generated messages and ensuring that they are stored and returned properly.
- **Censorship Feature**: Filtering out inappropriate words to ensure the messages remain appropriate.
- **Database Integration**: Managing the messages in the database, allowing for operations such as creating, reading, updating, and deleting messages.


# Input/Output: Demonstrating API Requests

Let's demo how we can interact with our chat application using **API requests**. We will use both the **frontend** and **Postman** to show how messages are sent, stored, and retrieved from the backend.

### Frontend API Request Example

This HTML form collects user input and sends a POST request to the Flask backend. The backend will process the message, censor inappropriate content, and store it in the database.

```html
<form id="chatForm">
    <input type="text" id="messageInput" placeholder="Enter your message">
    <button type="submit">Send Message</button>
</form>

<img alt="Chatroom running in frontend" src="/rowan_2025/images/frontend.png">


<script>
    document.getElementById('chatForm').addEventListener('submit', async function(event) {
        event.preventDefault();
        const message = document.getElementById('messageInput').value;
        const response = await fetch('http://localhost:8887/chat', {
            method: 'POST',
            headers: {
                'Content-Type': 'application/json'
            },
            body: JSON.stringify({ message: message, user_id: 1 })
        });
        const data = await response.json();
        console.log(data);
    });
</script>
```

### **What happens here?**
- The user inputs a message, and when the form is submitted, the JavaScript fetches the message and sends it as a JSON body to the Flask backend.
- The backend processes the message, censors any inappropriate content, and stores the message in the database.

---

# Postman API Request Example

In Postman, we can test the **POST** request to the `/chat` endpoint.

**Method**: `POST`  
**URL**: `http://localhost:8887/chat`  

**Body (JSON format)**:

```json
{
    "message": "This is an inappropriate message.",
    "user_id": 1
}
```

<img alt="Censored text in action" src="/rowan_2025/images/censorpy.png">


The backend will process the message, censor any inappropriate content, and respond with a JSON object.

**API Response (JSON format)**:

```json
{
    "message": "Message sent successfully",
    "original_message": "This is an inappropriate message.",
    "censored_message": "This is an ************ message."
}
```

### **What happens here?**
- We test the API in Postman by sending a POST request with a message to the `/chat` endpoint.
- The API processes the message, returns a success message, and includes both the original and censored versions of the message.


---

# Database Operations: Using db_init, db_restore, db_backup

To demonstrate how the database handles data creation and recovery, we use functions such as **db_init**, **db_restore**, and **db_backup**.

### **db_init**: Create Tester Data

This function initializes the database and adds test data to the `censor` table.

```python
# db_init.py
def db_init():
    """Create the database and add tester data"""
    with app.app_context():
        db.create_all()
        
        c1 = Censor(name='John Doe', uid='johndoe123', submission_text='Test message', censored_text='Test message', submission_date=date.today(), flagged_words='')
        db.session.add(c1)
        db.session.commit()
```

### **db_restore**: Restore Data to Original State

This function drops all tables and calls `db_init` to restore the original state of the database.

```python
# db_restore.py
def db_restore():
    """Restore the database to the original state"""
    with app.app_context():
        db.drop_all()
        db_init()
```

### **db_backup**: Back Up Data

This function allows us to back up the current state of the database.

```python
# db_backup.py
def db_backup():
    """Backup the current database state"""
    with app.app_context():
        # Your backup logic here (e.g., export database to a file)
        pass
```

---

# Working with Lists and Dictionaries

We use **lists** and **dictionaries** to store and manipulate data in our API and database.

### **List of Rows**: Fetching Multiple Records

In the database, each record is represented as a row, and when we retrieve multiple records, they are returned as a list of rows. For example:

```python
chats = MusicChat.query.all()
```

This query returns all chat records as a list of rows, and we can iterate over this list to display each message.

### **Dictionaries for Columns**: Structuring Data

When fetching a specific record, each record is returned as a dictionary where the column names are the keys, and the column values are the corresponding values. For example:

```python
chat_data = chat.read()  # Returns a dictionary
```

Each message will be represented as a dictionary, which allows us to easily format the data for API responses.

---

# Formatting Response Data (JSON)

The response data from the API is formatted as JSON to ensure compatibility with frontend applications and other services. For example:

```python
@app.route('/chat', methods=['GET'])
def get_all_chats():
    chats = MusicChat.query.all()
    return jsonify([chat.read() for chat in chats]), 200
```

This endpoint returns a list of all chat messages, formatted as JSON.

---

# Queries from Database: Extracting Python Lists

Queries from the database return Python lists, which can be used to display or process data. For example:

```python
chat_history = MusicChat.query.filter(
    (MusicChat._user_id == user1) | (MusicChat._user_id == user2)
).all()
```

This query returns a list of chat messages between two users, which is then processed and returned as a JSON response.

---

# Working with Columns: CRUD Operations

The `Censor` class is designed to handle CRUD operations for database records. Each method (create, read, update, delete) interacts with the columns of the `censor` table.

### **Create Method**

```python
def create(self, inputs=None):
    try:
        db.session.add(self)
        db.session.commit()
        return self
    except IntegrityError:
        db.session.rollback()
        return None
```

The `create` method adds a new record to the database.

### **Read Method**

```python
def read(self):
    data = {
        "id": self.id,
        "name": self.name,
        "uid": self.uid,
        "submission_text": self.submission_text,
        "censored_text": self.censored_text,
        "submission_date": self.submission_date,
        "flagged_words": self.flagged_words
    }
    return data
```

The `read` method retrieves the data from the database and returns it as a dictionary.

### **Update Method**

```python
def update(self, inputs):
    self.submission_text = inputs.get("submission_text", self.submission_text)
    self.censored_text = inputs.get("censored_text", self.censored_text)
    self.flagged_words = inputs.get("flagged_words", self.flagged_words)
    db.session.commit()
```

The `update` method updates specific columns of a record in the database.

### **Delete Method**

```python
def delete(self):
    db.session.delete(self)
    db.session.commit()
```

The `delete` method removes a record from the database.

<img alt="Chats after being deleted" src="/rowan_2025/images/chatnotfound.png">


---

# Algorithmic Code for API Requests

We will now demonstrate the code for handling an API request. This includes the use of **sequencing**, **selection**, and **iteration** within the code.

### **Handling API Request**

```python
@app.route('/chat', methods=['POST'])
def send_message():
    data = request.json
    message = data["message"]
    user_id = data["user_id"]

    # Censor the message before saving it
    censored_message = censor_message(message)
    chat = MusicChat(message=message, censored_message=censored_message, user_id=user_id)
    chat.create()

    return jsonify({
        "message": "Message sent successfully",
        "original_message": message,
        "censored_message": censored_message
    }), 200
```


### **What happens here?**
- This code listens for a **POST** request at the `/chat` endpoint, processes the message, and stores it in the database after censoring.
- The sequence of actions involves receiving the data, selecting the relevant message field, and iterating over the database to store the information.



## MCQ Reflection
 As a whole, my score has stayed the same. The first MCQ I got a 47/66 (71%), and this time I got a 48/67 (71%). Both of these could be in the range for a 5 on the AP exam. 

### Areas of Struggle: 
- Identifying and correcting errors (57% accuracy)
- Extracting information and data (43% accuracy)
- Variables and Assignments (0% accuracy)
- Iteration (60% accuracy)
- Developing Algorithms (75% Accuracy)
- Calling Procedures (60% Accuracy)
- Random Values (0% Accuracy)
- Algorithmic Efficiency (0% Accuracy)
- Fault Tolerance (0% Accuracy)

### Areas of Improvement: 
- Binary Numbers (75% --> 100%)
- Data Compression (33% --> 100%)


## Going Forward

As I move toward a career in architecture, computer science principles will be essential in enhancing design efficiency, sustainability, and innovation. From automation to AI-driven tools, integrating technology will allow me to push the boundaries of traditional architecture.  

### Key Applications  
- **Parametric & 3D Modeling** – Use programming (Python, Grasshopper) to automate design adjustments.  
- **Simulation & Data Analysis** – Apply computational tools to optimize environmental factors (sunlight, wind, energy efficiency).  
- **AI & Automation** – Explore generative design and AI-driven urban planning for smarter architecture.  
- **Problem-Solving Mindset** – Use computational thinking to break down design challenges logically.  

### Action Items  
✅ **Learn scripting for architectural software** (Grasshopper, Python for Rhino).  
✅ **Explore AI tools** for architecture and urban planning.  
✅ **Develop skills in data visualization and simulation** for sustainable design.  
✅ **Apply automation techniques** to streamline repetitive tasks in modeling.  